### LogisticRegression - breastcancer
- 로지스틱 회귀: S자 함수 사용하여 참 거짓(0 1) 분류: 이진분류에 많이 사용함
- 시그모이드 함수: 로지스텍 회귀에 사용. 값이 0과 1 시아에 있는 S자 함수 사용함.
- 공부 시간별 점수는 선형회귀, 공부 시간 별 합격 여부는 로지스틱 회귀 사용이 알맞음
- logit: logistic + probit
- 선형회귀 모델: 실제 값과 예측 값의 "오차"에 기반한 지표 사용. 종속 변수가 숫자 연속형
- 로지스틱 회귀 모델: 이진 분류 결과 평가를 위해 "오차 행렬"에 기반한 성능 지표인 정밀도, 재현율, F1 스코어, ROC_AUC 사용. 종속 변수가 이진...
- 오차행렬: 0일 때 0, 1일 때 1 예측하면 ㅇㅋ, 근데 그 반대면 No...

## 데이터 준비

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.datasets import load_breast_cancer

b_cancer = load_breast_cancer()
print(b_cancer.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [3]:
# b_cancer의 data 속성만 뽑아서 데이터 프레임화
b_cancer_df = pd.DataFrame(b_cancer.data, columns = b_cancer.feature_names)
# 'diagnosis(진단)' 속성 컬럼을 추가하기
# 이런 데이터 가진 사람은 유방암 안 걸림(0)/걸림(1)
b_cancer_df['diagnosis']= b_cancer.target
b_cancer_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [4]:
print('유방암 진단 데이터셋 크기 : ', b_cancer_df.shape)

유방암 진단 데이터셋 크기 :  (569, 31)


In [5]:
b_cancer_df.info()
# 컬럼 수가 31
# 데이터 타입은 모두가 플로트, 널값이 하나도 없구나

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [6]:
# 도메인 값이 어떤 건 0.0000002이러고 어떤 건 184.5, 1001 이럼
# 값끼리 너무 차이 나는데 이거 해결법 
# 이 데이터는 평균이 0이고 분산이 시그마인 정규 분포값으로 다시 배치하는 방법: sklearn.preprocessing 라이브러리의 StandardScaler 사용
# => 데이터 비슷하게 해주기
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
b_cancer_scaled = scaler.fit_transform(b_cancer.data) # 숫자 일괄화(?)
print(b_cancer.data[0])   # 표준 분표로 정리
print(b_cancer_scaled[0]) # 비슷한 거로? 정리..ㅋ

[1.799e+01 1.038e+01 1.228e+02 1.001e+03 1.184e-01 2.776e-01 3.001e-01
 1.471e-01 2.419e-01 7.871e-02 1.095e+00 9.053e-01 8.589e+00 1.534e+02
 6.399e-03 4.904e-02 5.373e-02 1.587e-02 3.003e-02 6.193e-03 2.538e+01
 1.733e+01 1.846e+02 2.019e+03 1.622e-01 6.656e-01 7.119e-01 2.654e-01
 4.601e-01 1.189e-01]
[ 1.09706398 -2.07333501  1.26993369  0.9843749   1.56846633  3.28351467
  2.65287398  2.53247522  2.21751501  2.25574689  2.48973393 -0.56526506
  2.83303087  2.48757756 -0.21400165  1.31686157  0.72402616  0.66081994
  1.14875667  0.90708308  1.88668963 -1.35929347  2.30360062  2.00123749
  1.30768627  2.61666502  2.10952635  2.29607613  2.75062224  1.93701461]


## 데이터 분석

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split # 학습, 테스트 데이터 나누기

# X, Y 설정하기
Y = b_cancer_df['diagnosis'] # 결과값은 유방암 걸린 거 유무, Series
X = b_cancer_scaled  # 속성들, # Array of float64
# 학습 0.7, 테스트0.3으로 데이터 나누기
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
# Array of float64, Array of float64, Series, Series

# 모델 생성
lr_b_cancer = LogisticRegression()
# 모델 학습
lr_b_cancer.fit(X_train, Y_train)
# 테스트: 예측 하기
Y_predict = lr_b_cancer.predict(X_test) # Array of int32

In [8]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
# 이진 분류 결과 평가를 위해 "오차 행렬"에 기반한 성능 지표인 정밀도, 재현율, F1 스코어, ROC_AUC 사용.

confusion_matrix(Y_test, Y_predict)
# 예측 클래스    0  1
# 실제 클래스 0: 00 01
# 실제 클래스 1: 10 11
# 암이 아닌 게 60, 암 아닌데 암이 암이라고 한 거 3
# 암인데 암이 아니라한 거 1개, 암인데 암이라 한 거 107개
# 꽤나 정확

array([[ 60,   3],
       [  1, 107]])

### 오차 행렬 필기

In [13]:
# 예측   0  1
# 실제 0 TN FP
# 실제 1 FN TP

# 정밀도 = TP / (FP + TP): 예측이 1인데 실제도 1인 것, 예측한 것들 중 실제로 맞는 률
#      = 실제1 예측1 / (예측1이고 실제가 0, 1인 거)
# 재현율(TPR) = TP / (FN + TP): 실제 1중에 예측도 1인 것 # 코로나 검사는 이게 중요
#       = 실제1 예측1 / (실제1이고 예측은 0, 1인 거)

# 센서 값 낮추면 실제 1인데 예측 0인 거가 실제 0이 예측1이 될 수 있는데, 그러면 실제 0인 것도 예측 1이 될 수 있음
# == 민감도 높이면 정밀도가 낮아짐.

# F1: 조화 평균: 정밀도와 재현률을 결합한 평가 지표

# ROC
# FPR가 높아질 때 TRP도 높아지긴 하는데 천천히 높아지면 좋은 거(1번), 빨리 높아지면 안 좋은 거(2번)
# 1번: 교재에서 둥근 그림 그림, 2번: 교재에서 x=y같은 선형 그래프
# 1번 면적이 높을 수록 좋다.
# 1에 가까울 수록 좋은 성능, 정확도 높다. 0.5에 가까울 수록 성능 나쁨.

# 정확도, 정밀도, 재현율, F1, ROC 다 높을 수록 좋다.

# 정리
# 파란 부분에서 빨간 부분의 비율
# 1: 정확도: 전체에서 실제 0일 때 예측 0, 실제 1일 때 예측도 1인 경우
# 2: 정밀도: 예측한 것이 실제와 맞는 경우
# 3: 재현율: 실제 
# 4: FPR: 실제 0일 때 예측이 1인 경우
# 5: TPR: 실제 1일 때 예측이 1인 경우 


In [12]:
acccuracy = accuracy_score(Y_test, Y_predict)
precision = precision_score(Y_test, Y_predict)
recall = recall_score(Y_test, Y_predict)
f1 = f1_score(Y_test, Y_predict)
roc_auc = roc_auc_score(Y_test, Y_predict)

print('정확도: {0:.3f}, \n정밀도: {1:.3f}, \n재현율: {2:.3f}, \nF1: {3:.3f}'.format(acccuracy,precision,recall,f1))
print('ROC_AUC: {0:.3f}'.format(roc_auc))

정확도: 0.977, 
정밀도: 0.973, 
재현율: 0.991, 
F1: 0.982
ROC_AUC: 0.972


### 연습문제1
- 정밀도: 3/5
- 재현율: 3/7

### 연습문제2
- 94 23
- 24 100
- 전체: 94+147=241
1. 참 양성 24, 거짓 음성 94, 참 음성 100, 거짓 양성 23
2. 정확도: 194/241
3. 정밀도
4. 오류율
5. 민감도
6. 특이도
7. 거짓 긍정, 거짓 부정이 있는데 위음성 오류율 FN/TP+